In [ ]:
# pyspark \
#   --packages io.delta:delta-core_2.12:2.3.0 \
#   --conf "spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension" \
#   --conf "spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog"

In [1]:
from pyspark.sql import SparkSession
from delta.tables import DeltaTable
from pyspark.sql.functions import col

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
No module named 'delta'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'delta'



In [2]:
data = [
    (1, "Alice", 30, "2025-12-01"),
    (2, "Bob",   28, "2025-12-01"),
    (3, "Cleo",  35, "2025-12-02"),
]
cols = ["id", "name", "age", "dt"]

df = spark.createDataFrame(data, cols)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df.write.format("delta").mode("overwrite").partitionBy("dt").saveAsTable("my_glue_db.delta_exp_tbl")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o85.saveAsTable.
: java.lang.ClassNotFoundException: 
Failed to find data source: delta. Please find packages at
https://spark.apache.org/third-party-projects.html
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedToFindDataSourceError(QueryExecutionErrors.scala:574)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:675)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:725)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:864)
	at org.apache.spark.sql.DataFrameWriter.saveAsTable(DataFrameWriter.scala:562)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at 

In [3]:
source_df = spark.createDataFrame([
    (1, "Alice", 30),
    (2, "Bob", 28),
    (4, "Dave", 35)
], ["id","name","age"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
source_df.write.format("delta").mode("overwrite").saveAsTable("my_glue_db.delta_src")

In [ ]:
if not DeltaTable.isDeltaTable(spark, spark.sql("DESCRIBE DETAIL my_glue_db.delta_src").collect()[0]['location']):
    target_initial = spark.createDataFrame([
        (1, "Alice", 29),   # age changed
        (2, "Bob", 28),
        (3, "Charlie", 40)
    ], ["id","name","age"])
    target_initial.write.format("delta").mode("overwrite").saveAsTable("my_glue_db.delta_src_tbl")

In [ ]:
delta_table = DeltaTable.forPath(spark, spark.sql("DESCRIBE DETAIL my_glue_db.delta_src").collect()[0]['location'])

In [ ]:
delta_table.alias("t").merge(source_df.alias("s"), "t.id = s.id").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

In [ ]:
spark.sql("select * from my_glue_db.delta_src").show()

In [ ]:
new_source_df = spark.createDataFrame([
    (1, "Alice", 50),
    (2, "Bob", 15),
    (5, "Rey", 25),
    (7, "Rashi", 30)
], ["id","name","age"])

In [ ]:
delta_table.alias("t").merge(new_source_df.alias("s"), "t.id = s.id").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

In [ ]:
spark.sql("select * from my_glue_db.delta_src").show()

In [1]:
delta_table.delete("true") # deletes all data files but keeps metadata

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'delta_table' is not defined
Traceback (most recent call last):
NameError: name 'delta_table' is not defined



In [ ]:
spark.sql("DROP TABLE IF EXISTS my_glue_db.delta_src")
spark.sql("DROP TABLE IF EXISTS my_glue_db.delta_exp_tbl")